# Imports

In [ ]:
# Configs
%load_ext autoreload
%autoreload 3
## other standard packages
import sys
## Env variables and preparation stuffs
sys.path.insert(0, "../")
from src_code.data_utils.dataset import GridDataset
from src_code.data_utils.dataset_utils import CellType

# Dataset

In [ ]:
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")

img_rgb, ascii_inp, path = dataset[0]

In [ ]:
img_rgb

In [ ]:
print(ascii_inp)

In [ ]:
print(f"{path = }")

# 3. VLM Inference

## 3.1 Open Flamingo
Source: https://github.com/mlfoundations/open_flamingo

In [ ]:
from open_flamingo import create_model_and_transforms
model, image_processor, tokenizer = create_model_and_transforms(
    clip_vision_encoder_path="ViT-L-14",
    clip_vision_encoder_pretrained="openai",
    lang_encoder_path="anas-awadalla/mpt-1b-redpajama-200b",
    tokenizer_path="anas-awadalla/mpt-1b-redpajama-200b",
    cross_attn_every_n_layers=1
)

In [ ]:
# grab model checkpoint from huggingface hub
from huggingface_hub import hf_hub_download
import torch

checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-3B-vitl-mpt1b", "checkpoint.pt")
model.load_state_dict(torch.load(checkpoint_path), strict=False)

In [ ]:
from PIL import Image
import requests
import torch

"""
Step 1: Load images
"""
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)

demo_image_two = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028137.jpg",
        stream=True
    ).raw
)

query_image = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028352.jpg", 
        stream=True
    ).raw
)


"""
Step 2: Preprocessing images
Details: For OpenFlamingo, we expect the image to be a torch tensor of shape 
 batch_size x num_media x num_frames x channels x height x width. 
 In this case batch_size = 1, num_media = 3, num_frames = 1,
 channels = 3, height = 224, width = 224.
"""
vision_x = [image_processor(demo_image_one).unsqueeze(0), image_processor(demo_image_two).unsqueeze(0), image_processor(query_image).unsqueeze(0)]
vision_x = torch.cat(vision_x, dim=0)
vision_x = vision_x.unsqueeze(1).unsqueeze(0)

"""
Step 3: Preprocessing text
Details: In the text we expect an <image> special token to indicate where an image is.
 We also expect an <|endofchunk|> special token to indicate the end of the text 
 portion associated with an image.
"""
tokenizer.padding_side = "left" # For generation padding tokens should be on the left
lang_x = tokenizer(
    ["<image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>An image of"],
    return_tensors="pt",
)


"""
Step 4: Generate text
"""
generated_text = model.generate(
    vision_x=vision_x,
    lang_x=lang_x["input_ids"],
    attention_mask=lang_x["attention_mask"],
    max_new_tokens=20,
    num_beams=3,
)

print("Generated text: ", tokenizer.decode(generated_text[0]))

In [ ]:
from PIL import Image
import requests
import torch
import numpy as np
"""
Step 1: Load images
"""
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")

img_rgb1, ascii_inp, path1 = dataset[0]
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")
img_rgb2, ascii_inp, path2 = dataset[1]
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")
img_rgb3, ascii_inp, path3 = dataset[2]


In [ ]:
img_rgb1

In [ ]:
img_rgb2

In [ ]:
img_rgb3.size

In [ ]:
path1

In [ ]:
path2

In [ ]:
path3

In [ ]:
"""
Step 2: Preprocessing images
Details: For OpenFlamingo, we expect the image to be a torch tensor of shape 
 batch_size x num_media x num_frames x channels x height x width. 
 In this case batch_size = 1, num_media = 3, num_frames = 1,
 channels = 3, height = 224, width = 224.
"""
vision_x = [image_processor(img_rgb1).unsqueeze(0), image_processor(img_rgb2).unsqueeze(0), image_processor(img_rgb3).unsqueeze(0)]
vision_x = torch.cat(vision_x, dim=0)
vision_x = vision_x.unsqueeze(1).unsqueeze(0)
print(vision_x.shape)
"""
Step 3: Preprocessing text
Details: In the text we expect an <image> special token to indicate where an image is.
 We also expect an <|endofchunk|> special token to indicate the end of the text 
 portion associated with an image.
"""
tokenizer.padding_side = "left" # For generation padding tokens should be on the left

lang_x = tokenizer(
    [f"<image> the action sequence required to move from the red cell to the green cell is {path1}<|endofchunk|><image> the action sequence required to move from the red cell to the green cell is {path2}<|endofchunk|><image> the action sequence required to move from the red cell to the green cell is "],
    return_tensors="pt",
)


"""
Step 4: Generate text
"""
generated_text = model.generate(
    vision_x=vision_x,
    lang_x=lang_x["input_ids"],
    attention_mask=lang_x["attention_mask"],
    max_new_tokens=20,
    num_beams=3,
)

print("Generated text: ", tokenizer.decode(generated_text[0]))

# What we tried.
1. directly prompt - garbage answer
2. one shot propmt - ???
3. two shot prompt - doesnt work
4. different image resolutions
5. ascii
6. bigger models??